In [ ]:
import cv2
import math
import time 
import sys
import glob
import imutils
import numpy as np
import sympy as sym
from scipy.special import comb
import matplotlib.pyplot as plt
from imutils.video import VideoStream

Top-view transformation function, perserving aspect ratio. The function is explained in the main notebook.

In [ ]:
def warp_ar_perserved(corner_points, image, ratio):
    ''' Compute the transformation matrix
    @ corner_points : 4 corner points selected from the image
    returns output : New warped image '''

    ''' Modified to preserve aspect ratio of the warped surface. The function is
    not used in below script but is kept here for reference. '''

    ''' the input parameter 'ratio' is the real life Width/Height ratio of the 
    cardboard '''

    # Create an array out of the 4 corner points
    pts1 = np.float32(corner_points)
    
    [tl,tr,br,bl] = pts1

    # Get the cardboard's pixel height and width in a manner that perserves 
    # aspect ratio
    cardH = math.sqrt(((br[0] - tr[0]) ** 2) + ((br[1] - tr[1]) ** 2))
    cardW = ratio * cardH;

    # fix the top left corner point of the Cardboard, and extract the new
    # corners using cardH and cardW
    pts2 = np.float32([[tl[0],tl[1]], [tl[0]+cardW, tl[1]], [tl[0]+cardW, tl[1]+cardH], [tl[0], tl[1]+cardH]])

    M = cv2.getPerspectiveTransform(pts1,pts2)
    
    # Add offset size (~padding) to the output's height and width
    # since the warped output is not a perfect rectangle (for display reasons)
    offsetSize = 500
    transformed = np.zeros((int(cardW + offsetSize), int(cardH + offsetSize)), dtype=np.uint8);
    warped_image = cv2.warpPerspective(image, M, transformed.shape) 
    
    return warped_image

Find the corner points of the map using the trackbars

In [ ]:
vs = VideoStream(src=1).start()
time.sleep(1.0)

frame = vs.read()
h,w,_ = frame.shape

def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.moveWindow('Trackbars',1000,0)

cv2.createTrackbar('height','Trackbars',0,h,nothing)
cv2.createTrackbar('width','Trackbars',0,w,nothing)


while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 1000 pixels
    frame = vs.read()
    
    line_h = cv2.getTrackbarPos('height','Trackbars')
    line_w = cv2.getTrackbarPos('width','Trackbars')
    cv2.line(frame,(1,line_h),(w-1,line_h),(255,0,255),1)
    cv2.line(frame,(line_w,1),(line_w,h-1),(255,0,255),1)
    
    # show the output frame
    cv2.namedWindow("Frame")
    cv2.moveWindow("Frame", 0,0)
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


Apply top-view transformtion. Since the transformed output is padded with zero value pixels, use the trackbars to find how much we need to crop from the output such that we remove the extra paddings. These values are the h1, h2, w1, w2 variables defined in  the main code.

In [ ]:
vs = VideoStream(src=1).start()
time.sleep(1.0)

# configure tl,bl,tr,br from the trackbar values obtained from running the first cell.
# These are changed depending on the how we fix our camera.
tl = () # tuple (x,y)
bl = () # tuple (x,y)
tr = () # tuple (x,y)
br = () # tuple (x,y)

corner_points = (tl,tr,br,bl)
frame = vs.read()
birdy = warp_ar_perserved(corner_points,frame,(1310/990))
h,w,_ = birdy.shape

def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.moveWindow('Trackbars',800,0)

cv2.createTrackbar('height','Trackbars',0,h,nothing)
cv2.createTrackbar('width','Trackbars',0,w,nothing)


while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 1000 pixels
    frame = vs.read()
    

    #matrix, (maxWidth,maxHeight) = compute_perspective_transform(corner_points)
    #birds_eye_view = cv2.warpPerspective(frame,matrix, (maxWidth,maxHeight))
    birdy = warp_ar_perserved(corner_points,frame,(1310/990))
    #h,w,_ = birdy.shape
    
    line_h = cv2.getTrackbarPos('height','Trackbars')
    line_w = cv2.getTrackbarPos('width','Trackbars')
    cv2.line(birdy,(1,line_h),(w-1,line_h),(255,0,255),1)
    cv2.line(birdy,(line_w,1),(line_w,h-1),(255,0,255),1)
    
    # show the output frame
    cv2.namedWindow("Bird's Eye View")
    cv2.moveWindow("Bird's Eye View", 0,0)
    
    cv2.imshow("Bird's Eye View", birdy)
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

Use the below cell to take pictures of transformed / top-view projection of the map

In [ ]:
vs = VideoStream(src=1).start()
time.sleep(1.0)

# configure tl,bl,tr,br from the trackbar values obtained from running the first cell.
# These are changed depending on the how we fix our camera.
tl = () # tuple (x,y)
bl = () # tuple (x,y)
tr = () # tuple (x,y)
br = () # tuple (x,y)

# configure h1,h2,w1,w2 from the trackbar values obtained from running the first cell.
# These are changed depending on the how we fix our camera.
h1 =    # scalar
h2 =    # scalar
w1 =    # scalar
w2 =    # scalar

while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 1000 pixels
    frame = vs.read()
    
    corner_points = (tl,tr,br,bl)
    #matrix, (maxWidth,maxHeight) = compute_perspective_transform(corner_points)
    #cv2_warp = cv2.warpPerspective(frame,matrix, (maxWidth,maxHeight))
    birdy = warp_ar_perserved(corner_points,frame,(1310/990))
    #height,width
    birdy = birdy[h1:h2,w1:w2]
    #print(birdy.shape)
    
    # show the output frame
    cv2.namedWindow("Frame")
    cv2.moveWindow("Frame", 0,0)
    cv2.namedWindow("Bird's Eye")
    cv2.moveWindow("Bird's Eye", 700,0)
    
    cv2.imshow("Frame", frame)
    #cv2.imshow("Bird's Eye", birds_eye_view)
    cv2.imshow("Bird's Eye", birdy)
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):       
        image = birdy
        image2 = frame
        break
        
# save the last frame
cv2.imwrite("Map.png",image)
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

